<a href="https://colab.research.google.com/github/tomcoomer123/mysoginy_meme_detection_hinglish/blob/main/Mysoginy_English_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.connecting to google drive





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1.Importing all libraries neede to build the model


In [ ]:
!pip install --upgrade transformers
import os
import random
import json
import math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.applications import (
    VGG19,
    ResNet152,
    EfficientNetB4
)
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from transformers import (
    AutoTokenizer,
    TFAutoModel,
    TFAutoModelForSequenceClassification
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from PIL import Image



1.Extracting the dataset and analysing it


In [ ]:
DATASET_ROOT = DATASET_ROOT = "/content/drive/MyDrive/Datasets/English_hindi"
csv_path = os.path.join(DATASET_ROOT, "MIMIC2024.csv")
df = pd.read_csv(csv_path)
df['ExtractedText'] = df['ExtractedText'].fillna("")
df.isnull().sum()



,0
FileName,0
ExtractedText,0
Misogyny,0
Objectification,0
Prejudice,0
Humiliation,0


making the final dataset


In [ ]:
target_col = "Misogyny"
text_col = "ExtractedText"
df['image_path'] = df['FileName'].apply(
    lambda x: os.path.join(DATASET_ROOT, "Files", x)
)
df_final = pd.concat([df[target_col],df[text_col],df['image_path']],axis = 1)
df_final.head()

,Misogyny,ExtractedText,image_path
0,1,haldi wala doodh turmeric latte,/content/drive/MyDrive/Datasets/English_hindi/...
1,1,७OKA BOLO QDOUDDO,/content/drive/MyDrive/Datasets/English_hindi/...
2,0,Kuch Mahino baad ye bhi Maa ban jayegi! Taimur...,/content/drive/MyDrive/Datasets/English_hindi/...
3,1,Mujhe to kuch aur hi laga,/content/drive/MyDrive/Datasets/English_hindi/...
4,0,Teacher tum kal school kyu nhi aaye the Me kyu...,/content/drive/MyDrive/Datasets/English_hindi/...


Train test and split

1.   Training Data - 70%
2.   Testing data - 15%
3.   validation data - 15%



In [ ]:
train_df, temp_df = train_test_split(
    df_final,
    test_size=0.3,
    stratify=df_final[target_col],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df[target_col],
    random_state=42
)

print(len(train_df), len(val_df), len(test_df))


3537 758 759


Checking imbalance in data

In [ ]:
class_count = train_df[target_col].value_counts()
class_count

imbalance_ratio = class_count.max() / class_count.min()
imbalance_ratio

1.009659090909091

In [ ]:
X_train_text = train_df[text_col].astype(str).tolist()
X_val_text   = val_df[text_col].astype(str).tolist()
X_test_text  = test_df[text_col].astype(str).tolist()

y_train = train_df[target_col].astype(int).values
y_val   = val_df[target_col].astype(int).values
y_test  = test_df[target_col].astype(int).values
[X_train_text[0], y_train[0],X_val_text[0], y_val[0],X_test_text[0], y_test[0]]

['Relax guys Coke pi rhi h bechari',
 np.int64(1),
 'Girl With 8M followers BoyWith २०OM foll७wess',
 np.int64(1),
 'शराफत की किस्सा खत्म अब जैसी वैसे हम दुनिया टुनिया',
 np.int64(0)]

In [ ]:
@tf.keras.utils.register_keras_serializable()
class HFBackbone(tf.keras.layers.Layer):
    def __init__(self, model_name, **kwargs):
        super().__init__(**kwargs)
        self.model_name = model_name
        self.transformer = TFAutoModel.from_pretrained(model_name, use_safetensors=False)

        # Fine-tune only last 2 layers
        for layer in self.transformer.layers[:-2]:
            layer.trainable = False
        for layer in self.transformer.layers[-2:]:
            layer.trainable = True

    def call(self, inputs):
        input_ids, attention_mask = inputs
        return self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            training=True
        ).last_hidden_state

    def get_config(self):
        config = super().get_config()
        config.update({"model_name": self.model_name})
        return config

In [ ]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from sklearn.metrics import f1_score, accuracy_score

# ------------------ Pooling ------------------

class MeanMaxPooling(tf.keras.layers.Layer):
    def call(self, inputs):
        outputs, attention_mask = inputs
        mask = tf.cast(tf.expand_dims(attention_mask, -1), tf.float32)
        masked = outputs * mask

        mean_pool = tf.reduce_sum(masked, axis=1) / tf.reduce_sum(mask, axis=1)
        max_pool = tf.reduce_max(masked, axis=1)

        return tf.concat([mean_pool, max_pool], axis=1)



# ------------------ Evaluation ------------------

def evaluate_predictions(y_true, probs):
    preds = (probs > 0.5).astype(int)
    acc = accuracy_score(y_true, preds)
    f1  = f1_score(y_true, preds, average="macro")
    return acc, f1

# ------------------ Training Function ------------------
def train_evaluate_text_model(
    model_name,
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    epochs=30,          # 🔥 allow long training
    batch_size=16,
    max_len=128
):


    print(f"\n🚀 Training TEXT model: {model_name}\n")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize(texts):
        return tokenizer(
            list(texts),
            truncation=True,
            padding="max_length",
            max_length=max_len,
            return_tensors="tf"
        )

    train_enc = tokenize(X_train)
    val_enc   = tokenize(X_val)
    test_enc  = tokenize(X_test)

    # ------------------ Backbone ------------------

    transformer = TFAutoModel.from_pretrained(
        model_name,
        use_safetensors=False
    )

    # 🔥 Fine-tune only last 2 layers (safe + powerful)
    for layer in transformer.layers[:-2]:
        layer.trainable = False
    for layer in transformer.layers[-2:]:
        layer.trainable = True

    # ------------------ Model ------------------

    input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = HFBackbone(model_name)([input_ids, attention_mask])


    pooled = MeanMaxPooling()([sequence_output, attention_mask])

    # 🔹 Simple classifier head

    x = tf.keras.layers.Dense(256,activation="relu")(pooled)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128,activation="relu")(x)   # or use 64 here



    logits = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=logits
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),   # 🔥 slower & better
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    # ------------------ Early Stopping ------------------

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,             # stop if no improvement for 4 epochs
        restore_best_weights=True
    )

    # ------------------ Training ------------------

    history = model.fit(
        {
            "input_ids": train_enc["input_ids"],
            "attention_mask": train_enc["attention_mask"]
        },
        y_train,
        validation_data=(
            {
                "input_ids": val_enc["input_ids"],
                "attention_mask": val_enc["attention_mask"]
            },
            y_val
        ),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=1
    )

    # 🔹 Ensure minimum 10 epochs
    trained_epochs = len(history.history["loss"])
    print(f"\n🧠 Trained for {trained_epochs} epochs")

    # ------------------ Testing ------------------

    logits = model.predict(
        {
            "input_ids": test_enc["input_ids"],
            "attention_mask": test_enc["attention_mask"]
        },
        batch_size=batch_size
    ).squeeze()

    probs = tf.nn.sigmoid(logits).numpy()

    acc, f1 = evaluate_predictions(y_test, probs)

    print("\n✅ Final Test Results")
    print(f"Accuracy : {acc:.4f}")
    print(f"Macro F1 : {f1:.4f}")

    return model,acc, f1

In [ ]:
text_models = [
    "roberta-base",
    "google-bert/bert-base-uncased",
    "albert-base-v2",
    "google/muril-base-cased",
    ]

results = []

for model_name in text_models:
    print(f"\nEvaluating TEXT model: {model_name}")
    acc, f1 = train_evaluate_text_model(
            model_name,
            X_train_text, y_train,
            X_val_text, y_val,
            X_test_text, y_test,
            epochs=5,
            batch_size=16
        )

    results.append({
        "Model": model_name,
        "Accuracy": acc,
        "Macro_F1": f1
    })



Evaluating TEXT model: roberta-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initiali

Epoch 1/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 78s 192ms/step - accuracy: 0.5173 - loss: 0.6940 - val_accuracy: 0.5172 - val_loss: 0.6829
Epoch 2/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 28s 128ms/step - accuracy: 0.5128 - loss: 0.6822 - val_accuracy: 0.5211 - val_loss: 0.6788
Epoch 3/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 27s 124ms/step - accuracy: 0.5177 - loss: 0.6768 - val_accuracy: 0.5330 - val_loss: 0.6753
Epoch 4/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 29s 128ms/step - accuracy: 0.5254 - loss: 0.6717 - val_accuracy: 0.5343 - val_loss: 0.6719
Epoch 5/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 28s 125ms/step - accuracy: 0.5368 - loss: 0.6700 - val_accuracy: 0.5435 - val_loss: 0.6694
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 469ms/step

Evaluating TEXT model: google-bert/bert-base-uncased


Some layers from the model checkpoint at google-bert/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at google-bert/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 59s 192ms/step - accuracy: 0.5090 - loss: 0.6927 - val_accuracy: 0.5198 - val_loss: 0.6839
Epoch 2/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 31s 139ms/step - accuracy: 0.5104 - loss: 0.6827 - val_accuracy: 0.5172 - val_loss: 0.6764
Epoch 3/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 137ms/step - accuracy: 0.5250 - loss: 0.6724 - val_accuracy: 0.5158 - val_loss: 0.6715
Epoch 4/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 137ms/step - accuracy: 0.5317 - loss: 0.6662 - val_accuracy: 0.5317 - val_loss: 0.6694
Epoch 5/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 31s 138ms/step - accuracy: 0.5450 - loss: 0.6646 - val_accuracy: 0.5396 - val_loss: 0.6665
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 466ms/step

Evaluating TEXT model: albert-base-v2


Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Epoch 1/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 58s 188ms/step - accuracy: 0.5163 - loss: 0.7307 - val_accuracy: 0.5158 - val_loss: 0.6897
Epoch 2/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 136ms/step - accuracy: 0.5161 - loss: 0.6880 - val_accuracy: 0.5303 - val_loss: 0.6817
Epoch 3/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 134ms/step - accuracy: 0.5244 - loss: 0.6720 - val_accuracy: 0.5369 - val_loss: 0.6748
Epoch 4/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 134ms/step - accuracy: 0.5335 - loss: 0.6703 - val_accuracy: 0.5290 - val_loss: 0.6700
Epoch 5/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 135ms/step - accuracy: 0.5356 - loss: 0.6581 - val_accuracy: 0.5290 - val_loss: 0.6643
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 447ms/step

Evaluating TEXT model: google/muril-base-cased


Some layers from the model checkpoint at google/muril-base-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 60s 200ms/step - accuracy: 0.5128 - loss: 0.6929 - val_accuracy: 0.5026 - val_loss: 0.6930
Epoch 2/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 31s 138ms/step - accuracy: 0.4997 - loss: 0.6930 - val_accuracy: 0.5026 - val_loss: 0.6928
Epoch 3/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 136ms/step - accuracy: 0.4837 - loss: 0.6930 - val_accuracy: 0.5026 - val_loss: 0.6926
Epoch 4/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 136ms/step - accuracy: 0.5004 - loss: 0.6926 - val_accuracy: 0.5026 - val_loss: 0.6924
Epoch 5/5
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 137ms/step - accuracy: 0.5053 - loss: 0.6923 - val_accuracy: 0.5026 - val_loss: 0.6922
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 432ms/step


In [ ]:
text_results_df = pd.DataFrame(results)
text_results_df.sort_values(by="Macro_F1", ascending=False)



,Model,Accuracy,Macro_F1
0,roberta-base,0.591568,0.589715
2,albert-base-v2,0.581028,0.577329
1,google-bert/bert-base-uncased,0.575758,0.575137
3,google/muril-base-cased,0.527009,0.459639


Image unimodal testing


In [ ]:
X_train_img = train_df["image_path"].values   # full path to image
X_val_img   = val_df["image_path"].values
X_test_img  = test_df["image_path"].values

y_train = train_df["Misogyny"].values
y_val   = val_df["Misogyny"].values
y_test  = test_df["Misogyny"].values


In [ ]:
def evaluate_predictions(y_true, probs, threshold=0.5):
    y_pred = (probs >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    return acc, f1


In [ ]:
IMG_SIZE = 224

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label


def make_dataset(X, y, batch_size=32, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:
def train_evaluate_image_model(
    model_name,
    base_model_fn,
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    img_size=224,
    epochs=5,
    batch_size=32
):

    # Datasets
    train_ds = make_dataset(X_train, y_train, batch_size, shuffle=True)
    val_ds   = make_dataset(X_val, y_val, batch_size)
    test_ds  = make_dataset(X_test, y_test, batch_size)

    # Base model
    base_model = base_model_fn(
        input_shape=(img_size, img_size, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False   # 🔥 freeze backbone

    # Model head
    inputs = tf.keras.Input(shape=(img_size, img_size, 3))
    x = base_model(inputs, training=False)
    if len(x.shape) == 4:
      x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    logits = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs, logits)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(2e-4),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    # Train
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        verbose=1
    )

    # Test
    logits = model.predict(test_ds).ravel()
    probs = tf.sigmoid(logits).numpy()

    acc, f1 = evaluate_predictions(y_test, probs)

    return model,acc, f1


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import EfficientNetV2B0


# -------- Vision Transformer (VALID TF-HUB) --------
def vit_model_fn(input_shape, include_top=False, weights=None):
    vit_layer = hub.KerasLayer(
        "https://tfhub.dev/sayakpaul/vit_s16_fe/1",
        trainable=False
    )

    inputs = tf.keras.Input(shape=input_shape)

    x = tf.keras.layers.Lambda(
        lambda t: vit_layer(t, training=False)
    )(inputs)

    # Output: (batch, 384)
    return tf.keras.Model(inputs, x)




# -------- BiT Model (VALID) --------
def bit_model_fn(input_shape, include_top=False, weights=None):
    bit_layer = hub.KerasLayer(
        "https://tfhub.dev/google/bit/m-r50x1/1",
        trainable=False
    )

    inputs = tf.keras.Input(shape=input_shape)

    x = tf.keras.layers.Lambda(
        lambda t: bit_layer(t, training=False)
    )(inputs)

    # Output: (batch, 384)
    return tf.keras.Model(inputs, x)


In [ ]:
image_models = [
    #("BiT", bit_model_fn),
    #("ViT", vit_model_fn),
    #("InceptionV3", InceptionV3),
    ("EfficientNetV2", EfficientNetV2B0)
]

image_results = []

for name, model_fn in image_models:
    print(f"\nEvaluating IMAGE model: {name}")

    model,acc, f1 = train_evaluate_image_model(
        name,
        model_fn,
        X_train_img, y_train,
        X_val_img, y_val,
        X_test_img, y_test,
        epochs=5,
        batch_size=32
    )

    image_results.append({
        "model_attr":model,
        "Model": name,
        "Accuracy": acc,
        "Macro_F1": f1
    })



Evaluating IMAGE model: EfficientNetV2
Epoch 1/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 788s 6s/step - accuracy: 0.4980 - loss: 0.7046 - val_accuracy: 0.5026 - val_loss: 0.6957
Epoch 2/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 28s 198ms/step - accuracy: 0.5074 - loss: 0.7023 - val_accuracy: 0.5026 - val_loss: 0.6937
Epoch 3/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 40s 194ms/step - accuracy: 0.4948 - loss: 0.6977 - val_accuracy: 0.5026 - val_loss: 0.6934
Epoch 4/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 27s 195ms/step - accuracy: 0.5036 - loss: 0.6950 - val_accuracy: 0.5026 - val_loss: 0.6950
Epoch 5/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 27s 196ms/step - accuracy: 0.4982 - loss: 0.6957 - val_accuracy: 0.5026 - val_loss: 0.6934
24/24 ━━━━━━━━━━━━━━━━━━━━ 149s 6s/step


In [ ]:
image_results_df = pd.DataFrame(image_results)
print(image_results_df)


                                 model_attr           Model  Accuracy  \
0  <Functional name=functional, built=True>  EfficientNetV2  0.501976   

   Macro_F1  
0  0.334211  


In [ ]:
#image_results[0]['model_attr'].save('model_eff.h5')
model_err = tf.keras.models.load_model('/content/model_eff.h5',compile=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,247,507 (23.83 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

 Optimizer params: 2 (12.00 B)